<a href="https://colab.research.google.com/github/grandhigowthamramsai123/mrm/blob/main/svm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import files
uploaded=files.upload()

Saving titanic.csv to titanic (1).csv


In [5]:
df=pd.read_csv('titanic.csv')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [7]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [8]:
df['Age'].fillna(df['Age'].median(),inplace = True)

/tmp/ipython-input-1750550615.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].median(),inplace = True)


In [9]:
df = df.drop(['PassengerId','Cabin','Embarked','Name','Ticket'], axis=1, errors='ignore')


In [10]:
print(df.groupby('Sex')['Survived'].mean())


Sex
female    0.742038
male      0.188908
Name: Survived, dtype: float64


In [11]:
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1}).fillna(0).astype(int)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    int64  
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
dtypes: float64(2), int64(5)
memory usage: 48.9 KB


In [13]:
Train,Valid,Test=np.split(df.sample(frac=1),[int(0.75*len(df)),int(0.8*len(df))])

/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [14]:
def mean(Train, x):
    Train[x] = (Train[x] - Train[x].mean()) / Train[x].std()
    return Train[x]

In [15]:
Train['Fare']=mean(Train,'Fare')
Train['Age']=mean(Train,'Age')
Train['Parch']=mean(Train,'Parch')
Train['SibSp']=mean(Train,'Parch')

Train['Survived'] = Train['Survived'].replace(0, -1)

Test['Survived'] = Test['Survived'].replace(0, -1)
Test['Parch']=mean(Test,'Parch')
Test['SibSp']=mean(Test,'Parch')
Test['Fare']=mean(Test,'Fare')
Test['Age']=mean(Test,'Age')

In [16]:
def seperate(df):
  x=df.iloc[:, 1:]
  y=df.iloc[:,0]

  return x,y
train_x,train_y=seperate(Train)
valid_x,valid_y=seperate(Valid)
test_x,test_y=seperate(Test)

In [17]:
train_y.value_counts()

,count
Survived,
-1,416
1,252


In [18]:
test_y.value_counts()

,count
Survived,
-1,108
1,71


In [19]:
type(train_y)

pandas.core.series.Series

In [20]:
train_y.shape

(668,)

In [21]:
class svm:
  def __init__(self, X, Y):
        self.X = X
        self.Y = Y
        n, n_features = X.shape
        self.weights =np.zeros(n_features)
        self.bias = 0.0
  def train(self,X,Y, epochs, lr,lambd):
    X = self.X.to_numpy()
    Y = self.Y.to_numpy()
    print(X.shape)
    print(Y.shape)

    for epoch in range(epochs):
      n, n_features = X.shape
      for i in range(n):
        x = X[i]
        y = Y[i]

        condition = y * (np.dot(x, self.weights) + self.bias)

        if condition >= 1:
          self.weights = self.weights - lr * (2 * lambd * self.weights)
        else:
          self.weights= self.weights + lr * (y * x - 2 * lambd * self.weights)
          self.bias = self.bias + lr * y
  def predict(self, X):
    linear_output = np.dot(X, self.weights) +  self.bias
    y_pred = np.sign(linear_output)
    print(X.shape)
    return y_pred
  def test(self, ypre,Y):
    ypre = pd.Series(ypre).reset_index(drop=True)
    Y = pd.Series(Y).reset_index(drop=True)
    cm = pd.crosstab(pd.Series(Y, name="Actual"),
                 pd.Series(ypre, name="Predicted"))

    print(cm)
    print(ypre.shape)
    ypre.value_counts()
    print(Y.shape)
    Y.value_counts()






In [22]:
model = svm(train_x, train_y)
model.train(train_x, train_y,epochs=5000, lr=0.05,lambd=0.04)

preds = model.predict(test_x)
model.test(preds, test_y)

(668, 6)
(668,)
(179, 6)
Predicted  -1.0   1.0
Actual               
-1          103     5
 1           47    24
(179,)
(179,)
